In [1]:
import nltk
import sklearn
import pandas
import numpy as np
from nltk.probability import FreqDist, ConditionalFreqDist
from nltk.tokenize import RegexpTokenizer
from nltk.classify import MaxentClassifier
from nltk import classify
from sklearn import cross_validation
import re

### 1. Importamos el corpus de comentarios

In [2]:
comentarios = pandas.read_csv("comentarios_peliculas.csv", skiprows=1, delimiter=';', skip_blank_lines=True, encoding='utf-8')

### 1.1 Realizamos un breve analisis del corpus 

In [3]:

positivos = 0 
neutros = 0
negativos = 0
total = str(len(comentarios))
largo_comentario = 0
lista_comentarios = comentarios.values.tolist()
for x in lista_comentarios:
    if x[7] < 3:
        negativos += 1
    elif x[7] == 3:
        neutros += 1
    else:
        positivos += 1
    largo_comentario += len(re.findall(r'\w+', x[6]))


avg_positivos = str(float(positivos)*100 / float(total))
avg_negativos = str(float(negativos)*100 / float(total))
avg_neutros = str(float(neutros)*100 / float(total))
avg_largo_comentario = str(float(largo_comentario)/ float(total))
        
print ("El corpus tiene " + total + " comentarios")
print ("Hay " + str(positivos) + " comentarios positivos lo que equivale al  " + avg_positivos[:4] + "%")
print ("Hay " + str(neutros) + " comentarios neutros lo que equivale al  " + avg_neutros[:4] + "%")
print ("Hay " + str(negativos) + " comentarios negativos lo que equivale al  " + avg_negativos[:4] + "%")
print ("El largo de palabras promedio de los comentarios es " + avg_largo_comentario[:2])

El corpus tiene 1447 comentarios
Hay 859 comentarios positivos lo que equivale al  59.3%
Hay 222 comentarios neutros lo que equivale al  15.3%
Hay 366 comentarios negativos lo que equivale al  25.2%
El largo de palabras promedio de los comentarios es 38


### 2. Separamos entrenamiento y testeo, lo pasamos a listas comunes y eliminamos datos que no utilizamos. 

In [4]:
np.random.seed(42) 
msk = np.random.rand(len(comentarios)) < 0.8
comentarios_training= comentarios[msk] 
comentarios_testing = comentarios[~msk] 

lista_comentarios_training = comentarios_training.values.tolist()
for x in lista_comentarios_training:
    del x[8]
    del x[5]
    del x[4]
    del x[3]
    del x[2]
    del x[1]
    del x[0]
print ("El corpus de entrenamiento tiene largo " + str(len(lista_comentarios_training)))
lista_comentarios_testing = comentarios_testing.values.tolist()
for x in lista_comentarios_testing:
    del x[8]
    del x[5]
    del x[4]
    del x[3]
    del x[2]
    del x[1]
    del x[0]
print ("El corpus de testing tiene largo " + str(len(lista_comentarios_testing)))

El corpus de entrenamiento tiene largo 1147
El corpus de testing tiene largo 300


### 3. Obtenemos las palabras mas comunes del corpus, quitando puntuación y stopwords del Español

In [11]:
textoComentariosTraining = ""
for x in lista_comentarios_training:
    textoComentariosTraining += " " + x[0].lower()

tokenizer = RegexpTokenizer(r'\w+')
corpus_text = tokenizer.tokenize(textoComentariosTraining) 
stopwordsEspanol = nltk.corpus.stopwords.words('spanish')
stopwordsEspanol.remove("no")
stopwordsEspanol.remove("sí")
palabrasMasComunes = FreqDist(w.lower() for w in corpus_text if w not in stopwordsEspanol)

### 4. Utilizamos cross-validation para determinar el mejor N

In [12]:
def word_feats(commentWords): # FUNCION QUE DEVUELVE UN DICCIONARIO DE FEATURES, CON VALORES TRUE Y FALSE
    commentWords = tokenizer.tokenize(commentWords)
    return dict([(word[0], word[0] in commentWords) for word in palabras_features])

def classification(value): # FUNCION QUE TRANSFORMA UN NUMERO DE 1..5 A UNA CLASIFICACION
    if value > 3:
        classification = "negative"
    elif value == 3:
        classification = "neutro"
    else:
        classification = "positive"
    return classification

# VARIABLES DEL FOR. DETERMINAN EL TIEMPO DE EJECUCION EN GRAN MEDIDA
posiblesN = [10,40,70,100] 
k = 5 # Por defecto, se utiliza 10
max_iterations = 20 
listaAccuracy = [] 

for n in posiblesN:
    totalAccuracyN = 0
    palabras_features = palabrasMasComunes.most_common(n)
    array_for_training = [(word_feats(f[0]), classification(f[1])) for f in lista_comentarios_training]
    cv = cross_validation.KFold(len(array_for_training), k, None, False)
    for traincv, testcv in cv: 
        classifier = nltk.MaxentClassifier.train(array_for_training[traincv[0]:traincv[len(traincv)-1]], max_iter=max_iterations) 
        accuracyCV = nltk.classify.util.accuracy(classifier, array_for_training[testcv[0]:testcv[len(testcv)-1]])
        print ('accuracy:', accuracyCV)
        totalAccuracyN += accuracyCV
    listaAccuracy.append(totalAccuracyN/k)
    
print(listaAccuracy)

# SELECCIONAMOS EL MEJOR N
index = listaAccuracy.index(max(listaAccuracy)) # OBTENGO INDICE DEL MAYOR
N = posiblesN[index]

#para ver la accuracy contra testing, cosa que NO hay que hacer:::
#array_for_test = [(word_feats(f[0]), classification(f[1])) for f in lista_comentarios_testing]
#accuracyTesting = classify.accuracy(bestClassifierCV,array_for_test)

  ==> Training (20 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -1.09861        0.255
             2          -0.81285        0.591
             3          -0.80374        0.591
             4          -0.79574        0.591
             5          -0.78873        0.591
             6          -0.78258        0.591
             7          -0.77718        0.591
             8          -0.77242        0.591
             9          -0.76822        0.591
            10          -0.76450        0.594
            11          -0.76119        0.593
            12          -0.75823        0.594
            13          -0.75559        0.591
            14          -0.75322        0.598
            15          -0.75109        0.597
            16          -0.74917        0.597
            17          -0.74744        0.597
            18          -0.74586        0.598
            19          -0.74444        0.598
  

In [13]:
print(index)
print(N)

1
40
